# Baryonic Tully-Fisher Relation #

The goal of this notebook is to demonstrate how to calculate the baryonic tully-fisher distance for a galaxy in APPSS.  The BTF predicts the total baryonic mass based on the 21-cm line width.  If we have expressions for the stellar mass and HI mass in terms of the distance to the galaxy, then we can solve for distance.

In [3]:
import numpy as np

## Measure HI velocity width from HI spectrum (W$_{50}$) ##

The HI velocity width in units of km/s.

In [33]:
W50 = 100.

For convenience, I am going to list all of the other measured quantities that we need for each galaxy.  This will make it easier to calculate the distance for a different galaxy.  The values that I am using for this example are made up.

In [43]:
ba = 0.33 # observed axis ratio of galaxy 
flux_HI = .505 # HI flux in units of Jy-km/s
g = 15.29 # SDSS apparent g-band magnitude
i = 14.19 # SDSS apparent i-band magnitude

## Correct velocity width for inclination ##

We assume that the spiral galaxies is similar to a pancake; they are circular when viewed face on, and they have a much smaller dimension perpendicular to their face.  

The ellipticity that we observe is then due to the orientation of the galaxy.  We can express this observed elongation in terms of the axis ration of the galaxy, $b/a$.

By definition, the inclination angle is the angle between your line of sight and the area vector that is oriented perpendicular to the plane of the galaxy.  The inclination angle is $0^\circ$ for a face-on galaxy and $90^\circ$ for an edge-on galaxy.

We are using the relationship from Stark, McGaugh, & Swaters (2009) to relate the inclination angle to the observed axis ratio.

$$ sin i = \sqrt{\frac{1 - (b/a)^2}{1 - 0.15^2} }$$

In [44]:
inclination = np.arcsin((1-ba**2)/(1-.15**2))
print 'inclination = %.1f degrees'%(np.degrees(inclination))

inclination = 65.7 degrees


## Use BTF to get total baryonic mass ##

Use calibration from Stark, McGaugh, & Swaters (2009; http://adsabs.harvard.edu/abs/2009AJ....138..392S) to get the baryonic mass

$$\log_{10} M_b = 3.94 \log_{10} v_f + 1.79$$

where 

$$ M_b = M_\star + M_{gas} $$ 

and 

$$v_f = v_{obs}/sini $$

$v_{obs}$ is $W_{50}$.


In [45]:
vf = W50/np.sin((inclination))
log_Mb = 3.94 * np.log10(vf) + 1.79
print 'log(Mb) = %.2f'%(log_Mb)
Mb = 10.**log_Mb

log(Mb) = 9.83


## Get Expression for gas mass ##

The gas mass should include both the molecular and atomic gas, where the molecular gas is traced by CO, for example, and the atomic gas mass is traced by HI.  For now we will approximate the gas mass as this HI mass, but we should add a correction to account for the molecular gas at a later date.  


** HI Mass **

We calculate the HI mass as a function of distance from the HI line flux (Roberts 1962), according to the conversion given here http://egg.astro.cornell.edu/alfalfa/ugrad/himass.htm.

$$M_{HI} = 2.36 \times 10^5 \times F_{HI} \times d_{Mpc}^2$$

where $F_{HI}$ is the integrated flux from HI in units of Jy-km/s.


In [46]:
MHI_per_distance2 = 2.36e5 * flux_HI

## Correct SDSS Magnitudes for Galactic and Internal Redenning ##

We need to correct the observed SDSS magnitudes for galactic and internal redenning.  I haven't implemented this yet...

## Estimate Stellar Mass ##


Estimate the stellar mass using Taylor+2011 relation, express in terms of distance. (accuracy of 0.1 dex)  Taylor+2011 gives:

$$\log_{10} \left( \frac{M_\star}{M_\odot}\right) = 1.15 + 0.70 (g-i) - 0.4 M_i $$

and the distance modulus ($m - M = 5 \log_{10}d_{pc} - 5$) gives:

$$ M_i = m_i - 5 \log_{10} d_{pc} + 5 = i - 5 \log_{10} d_{pc} + 5 $$

We can combine the above two equations to get:

$$\log_{10} \left( \frac{M_\star}{M_\odot}\right) = 1.15 + 0.70 (g-i) - 0.4 (i - 5 \log_{10} d_{pc} + 5 ) $$

Distribute $-0.4$:
$$\log_{10} \left( \frac{M_\star}{M_\odot}\right) = 1.15 + 0.70 (g-i) - 0.4 i + 2 \log_{10} d_{pc} - 2  $$

Bring factor in front of log inside the log for distance term:
$$\log_{10} \left( \frac{M_\star}{M_\odot}\right) = 1.15 + 0.70 (g-i) - 0.4 i + \log_{10} (d_{pc}^2) - 2  $$



We want the distance to be in units of Mpc, so we can manipulate the $\log_{10} (d_{pc}^2)$ expression (I have a sign issue here):

$$\log_{10} (d_{pc}^2) = \log_{10} \left(d_{pc}^2 \left(\frac{1~Mpc}{10^6~pc}\right)^2\right)  = \log_{10} \left( \frac{d_{Mpc}^2}{10^{12}}\right) = \log_{10} \left({d_{Mpc}^2}\right) + 12 $$

Going back and substituting into stellar mass equation gives:
$$\log_{10} \left( \frac{M_\star}{M_\odot}\right) = \log_{10} \left({d_{Mpc}^2}\right) + 12 + 1.15 + 0.70 (g-i) - 0.4 i  - 2  $$


$$\log_{10} \left( \frac{M_\star}{M_\odot}\right) = \log_{10} \left({d_{Mpc}^2}\right) + 1.15 + 0.70 (g-i) - 0.4 i  + 10  $$


$$\frac{M_\star}{M_\odot} = d_{Mpc}^2 \times 10^{1.15 +0.7(g-i) - 0.4 i + 10}  $$


In [64]:
Mstar_per_distance2 = 10.**(1.15 + 0.7*(g-i) - 0.4*i +10)

## Combine Mass Equations ##

We can now combine the stellar mass and HI mass equations to get an expression for the baryonic mass in terms of distance.

$$ M_b = d_{Mpc}^2 \times 10^{1.15 +0.7(g-i) - 0.4 i - 14} + 2.36 \times 10^5 \times F_{HI} \times d_{Mpc}^2 $$ 

$$ M_b = d_{Mpc}^2 \left( 10^{1.15 +0.7(g-i) - 0.4 i - 14} + 2.36 \times 10^5 \times F_{HI} \right) $$

Solving for distance gives:

$$d_{Mpc} = \sqrt{ \frac{M_b}{10^{1.15 +0.7(g-i) - 0.4 i - 14} + 2.36 \times 10^5 \times F_{HI}}} $$

In [65]:
distance_Mpc = np.sqrt(Mb/(Mstar_per_distance2 + MHI_per_distance2))
print 'distance = %.2f (Mpc)' %(distance_Mpc)

distance = 59.97 (Mpc)


In [66]:
print Mstar_per_distance2
print MHI_per_distance2
print Mb

1753880.50184
119180.0
6735190287.81


## In One Fell Swoop ##

Here is all the code in two cells so you can calculate distances more quickly.  Of course, an even better idea is to write your own program.

In [67]:
W50 = 100.
ba = 0.33 # observed axis ratio of galaxy 
flux_HI = .505 # HI flux in units of Jy-km/s
g = 15.29 # SDSS apparent g-band magnitude
i = 14.19 # SDSS apparent i-band magnitude

In [63]:
inclination = np.arcsin((1-ba**2)/(1-.15**2))
print 'inclination = %.1f degrees'%(np.degrees(inclination))
vf = W50/np.sin((inclination))
log_Mb = 3.94 * np.log10(vf) + 1.79
print 'log(Mb) = %.2f'%(log_Mb)
Mb = 10.**log_Mb
MHI_per_distance2 = 2.36e5 * flux_HI
Mstar_per_distance2 = 10.**(1.15 + 0.7*(g-i) - 0.4*i +10)
distance_Mpc = np.sqrt(Mb/(Mstar_per_distance2 + MHI_per_distance2))
print 'distance = %.2f (Mpc)' %(distance_Mpc)

inclination = 65.7 degrees
log(Mb) = 9.83
distance = 59.97 (Mpc)
